In [81]:
import torch 
from torch import nn, optim
import torchvision 
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt 
import math 
import numpy as np
from PIL import Image
import cv2

In [3]:
class DigitRecognizer(nn.Module):
    def __init__(self, input_size, hidden_layers, num_classes):
        super(DigitRecognizer, self).__init__()
        self.input = nn.Linear(input_size, hidden_sizes[0])
        self.relu_0 = nn.ReLU()
        self.hidden1 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.relu_1 = nn.ReLU()
        self.output = nn.Linear(hidden_sizes[1], num_classes)
        
    def forward(self, X):
        model = self.input(X)
        model = self.relu_0(model)
        model = self.hidden1(model)
        model = self.relu_1(model)
        model = self.output(model)
        return model 

In [5]:
batch_size = 50
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,)),])
train_data = datasets.MNIST('./', download=True, train=True, transform=transform)
test_data = datasets.MNIST('./', download=True, train=False, transform=transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False , num_workers=2)

In [32]:
model = torch.load('digit_classifier.pt')
model.eval()

def eval_model(model, images):
    # images, labels are batches of 50
    images = images.reshape(-1, 784)
    outputs = model(images) 

    _, test_predicted = torch.max(outputs.data,1)
    return test_predicted

In [39]:
#adversarial criterion is untargeted
def generate_initial_adverarial(targeted_class):
    while True: 
        adversarial = np.random.randint(low = 0, high = 255, size = (50,28,28))
        model_output = eval_model(model, torch.tensor(adversarial, dtype = torch.float32)).item()
        if model_output != targeted_class: 
            return adversarial

In [76]:
def sample_proposal_dist(original_image, prev_image, shape, delta):
    new_pertubation = np.random.normal(size = shape)
    # normalize the pertubation
    new_pertubation /= np.linalg.norm(new_pertubation, 2)
    # scale its l2 norm as follows: ||n_k||_2 = delta * d(0,0^k-1)
    new_pertubation *= delta * np.linalg.norm(original_image - prev_image)
    perturbed_image = new_pertubation + prev_image
    return perturbed_image

In [ ]:
#The distance between points is defined as the norm of their difference.
def project_onto_sphere(original_image, pertubation):
    # radius of the circle
    orthogonal_vector = (original_image - pertubation).astype(np.float32)
    
    
    
    
    
    
    

In [40]:
# d(0,0^k-1) - d(0,0^k-1 + n^k) =  epsilon d(0,0^k-1)
def step_towards(original_image, perturbed_image, epsilon):
    return (original_image - perturbed_image).astype(np.float32) * epsilon